### Import File

In [1]:
from PIL import Image

### Interactive

In [2]:

from PIL import Image, ImageDraw, ImageOps
import ipywidgets as widgets
from IPython.display import display

class ImageGridProcessor:
    def __init__(self, image_path, border_width=5, background_color='black'):
        self.image_path = image_path
        self.border_width = border_width
        self.background_color = background_color
        self.colors = [(0, 0, 0), (255, 255, 255), (128, 128, 128)]  # Black, White, Grey
    
    def split_image_into_grid(self, grid_size):

        # Open the image
        image = Image.open(self.image_path)
        width, height = image.size
        
        height_width_min = min([width,height])
        self.square_dim = height_width_min
        
        # Calculate the size of each square
        square_width = height_width_min // grid_size
        square_height = height_width_min // grid_size
        
        # Split the image into grid squares
        grid_images = []
        for row in range(grid_size):
            for col in range(grid_size):
                left = col * square_width
                upper = row * square_height
                right = (col + 1) * square_width
                lower = (row + 1) * square_height
                grid_images.append(image.crop((left, upper, right, lower)))
        
        return grid_images, square_width, square_height
    
    @staticmethod
    def color_distance(c1, c2):
        return sum((a - b) ** 2 for a, b in zip(c1, c2)) ** 0.5
    
    def closest_color(self, pixel):
        distances = [self.color_distance(pixel, color) for color in self.colors]
        return self.colors[distances.index(min(distances))]
    
    def convert_to_closest_color(self, grid_images):
        converted_images = []
        self.grid_color_names = []
        self.grid_colors = []

        for img in grid_images:
            # Convert image to a small thumbnail to calculate average color
            img_thumbnail = img.resize((1, 1))
            avg_color = img_thumbnail.getpixel((0, 0))[0:3]

            # Round RBG (0, 171, 0) -> (0, 170, 0) to nearest 5 value to normalize color outputs
            normalize = 5
            avg_color = tuple(5 * round(value / normalize) for value in avg_color)

            # If color list is not supplied, default to average color of grid square
            if not self.colors:
                closest = avg_color
            else:            
                # Determine the closest color from the list
                closest = self.closest_color(avg_color)
            
            # Convert RGB color to named color 
            color_name = find_closest_color_name(closest)

            # Track closest colors to summarize 
            self.grid_color_names.append(color_name)
            self.grid_colors.append(closest)

            # Create a new image with the closest color
            new_img = Image.new('RGB', img.size, closest)
            converted_images.append(new_img)
        return converted_images
    
    def remerge_grid_images(self, grid_images, grid_size, square_width, square_height):
        # Calculate the size of the new image
        new_width = grid_size * (square_width + self.border_width) + self.border_width
        new_height = grid_size * (square_height + self.border_width) + self.border_width
        
        # Create a new image with the specified background color
        new_image = Image.new('RGB', (new_width, new_height), self.background_color)
        
        # Paste the grid images into the new image with the border
        for index, img in enumerate(grid_images):
            row = index // grid_size
            col = index % grid_size
            left = col * (square_width + self.border_width) + self.border_width
            upper = row * (square_height + self.border_width) + self.border_width
            new_image.paste(img, (left, upper))
        
        return new_image
    
    def process_and_display(self, grid_size):
        grid_images, square_width, square_height = self.split_image_into_grid(grid_size)
        converted_images = self.convert_to_closest_color(grid_images)
        merged_image = self.remerge_grid_images(converted_images, grid_size, square_width, square_height)
        
        display(merged_image)
        return merged_image

# Interactive widgets
image_path = r'image\python.PNG'

grid_size_slider = widgets.IntSlider(value=10, min=2, max=100, step=1, description='Grid Size:')

processor = ImageGridProcessor(image_path=image_path)

ui = widgets.VBox([grid_size_slider])
out = widgets.interactive_output(
    processor.process_and_display, 
    {'grid_size': grid_size_slider}
)

display(ui, out)

Output()

In [3]:
import matplotlib.colors as mcolors

def color_name_to_rgb(color_name):
    # Convert color name to RGB tuple
    try:
        rgb = mcolors.to_rgb(color_name)
        # Convert float values to integers (0-255)
        rgb = tuple(int(c * 255) for c in rgb)
        return rgb
    except ValueError:
        # Handle case where color_name is not valid
        raise ValueError(f"Invalid color name: {color_name}")
    
color_name_to_rgb("blue") 

(0, 0, 255)

In [10]:
import matplotlib.colors as mcolors
import math

# Create a dictionary mapping RGB values to color names
rgb_to_name = {tuple(int(c * 255) for c in mcolors.to_rgb(name)): name for name in mcolors.CSS4_COLORS}

def color_distance(c1, c2):
    return sum((a - b) ** 2 for a, b in zip(c1, c2)) ** 0.5

def find_closest_color_name(rgb):
    if not isinstance(rgb, tuple) or len(rgb) != 3 or not all(isinstance(c, int) and 0 <= c <= 255 for c in rgb):
        raise ValueError("RGB must be a tuple of three integers in the range 0-255")

    closest_color = min(rgb_to_name.keys(), key=lambda color: color_distance(rgb, color))
    return rgb_to_name[closest_color]

# Example usage
rgb_value = (0, 0, 210)  # RGB value close to "dodgerblue"
closest_color_name = find_closest_color_name(rgb_value)
print(closest_color_name)

mediumblue


## Words on Grid

In [23]:
import math

def format_to_grid(flat_list):
    # Calculate the grid dimensions
    size = int(math.sqrt(len(flat_list)))
    if size * size != len(flat_list):
        raise ValueError("The size of the flat list is not a perfect square.")
    
    grid = []
    index = 0
    for row in range(size):
        grid_row = []
        for col in range(size):
            grid_row.append((flat_list[index], ''))
            index += 1
        grid.append(grid_row)
    
    return grid

grid_color = format_to_grid(flat_list=processor.grid_index_color)

In [25]:
grid_color[0]

[((255, 255, 255), ''),
 ((255, 255, 255), ''),
 ((128, 128, 128), ''),
 ((0, 0, 0), ''),
 ((0, 0, 0), ''),
 ((0, 0, 0), ''),
 ((0, 0, 0), ''),
 ((128, 128, 128), ''),
 ((255, 255, 255), ''),
 ((255, 255, 255), '')]

In [26]:
data = [((255, 255, 255), ''),
        ((255, 255, 255), ''),
        ((128, 128, 128), ''),
        ((0, 0, 0), ''),
        ((0, 0, 0), ''),
        ((0, 0, 0), ''),
        ((0, 0, 0), ''),
        ((128, 128, 128), ''),
        ((255, 255, 255), ''),
        ((255, 255, 255), '')]

result = []
current_tuple = None
current_count = 0

for item in data:
    if current_tuple is None:
        current_tuple = item[0]
        current_count = 1
    elif current_tuple == item[0]:
        current_count += 1
    else:
        result.append([current_tuple, current_count])
        current_tuple = item[0]
        current_count = 1

# Append the last counted tuple
if current_tuple is not None:
    result.append([current_tuple, current_count])

print(result)

[[(255, 255, 255), 2], [(128, 128, 128), 1], [(0, 0, 0), 4], [(128, 128, 128), 1], [(255, 255, 255), 2]]


In [15]:
def create_grid(rows, cols, default_color=(255, 255, 255)):
    return [[(default_color, '') for _ in range(cols)] for _ in range(rows)]

def can_place_word(grid, word, color, start_row, start_col, direction):
    rows, cols = len(grid), len(grid[0])
    length = len(word)

    if direction == 'horizontal':
        if start_col + length > cols:
            return False
        for i in range(length):
            cell_color, cell_letter = grid[start_row][start_col + i]
            if cell_color != color and cell_letter != '':
                return False
            if cell_letter != '' and cell_letter != word[i]:
                return False
    elif direction == 'vertical':
        if start_row + length > rows:
            return False
        for i in range(length):
            cell_color, cell_letter = grid[start_row + i][start_col]
            if cell_color != color and cell_letter != '':
                return False
            if cell_letter != '' and cell_letter != word[i]:
                return False
    return True

def place_word(grid, word, color, start_row, start_col, direction):
    length = len(word)
    if direction == 'horizontal':
        for i in range(length):
            grid[start_row][start_col + i] = (color, word[i])
        return grid
    elif direction == 'vertical':
        for i in range(length):
            grid[start_row + i][start_col] = (color, word[i])
        return grid

def find_best_placement(grid, word, color):
    best_position = None
    max_overlap = 0
    directions = ['horizontal', 'vertical']
    
    for direction in directions:
        for row in range(len(grid)):
            for col in range(len(grid[0])):
                if can_place_word(grid, word, color, row, col, direction):
                    overlap = 0
                    if direction == 'horizontal':
                        for i in range(len(word)):
                            if grid[row][col + i][1] == word[i]:
                                overlap += 1
                    elif direction == 'vertical':
                        for i in range(len(word)):
                            if grid[row + i][col][1] == word[i]:
                                overlap += 1
                    if overlap >= max_overlap:
                        max_overlap = overlap
                        best_position = [row, col, direction]
    
    return best_position

def print_grid(grid):
    for row in grid:
        print(' '.join(['_' if cell[1] == '' else cell[1] for cell in row]))

# Example usage
#grid = create_grid(10, 10)
grid = grid_color

words_with_colors = [("hi", (255, 255, 255)), ("world", (0, 255, 0)), ("hold", (255, 0, 0))]

for word, color in words_with_colors:
    best_position = find_best_placement(grid, word, color)
    if best_position:
        grid = place_word(grid, word, color, best_position[0], best_position[1], best_position[2])

#print_grid(grid)


In [16]:
def flatten_grid(grid):
    flat_list = []
    for row in grid:
        for cell in row:
            flat_list.append(cell[1])  # Append the color tuple
    return flat_list

grid_letters = flatten_grid(grid)

In [17]:
from PIL import Image, ImageDraw, ImageFont


def add_words_to_images(grid_images, grid_letters):
    word_images = []
    
    for index_image, index_letter in zip(grid_images, grid_letters):
        # Create a draw object
        draw = ImageDraw.Draw(index_image)

        # Define the text and font
        text = index_letter

        # Calculate text size and position using textbbox
        image_width, image_height = index_image.size
        text_x = (image_width) // 2
        text_y = (image_height) // 2

        font = ImageFont.truetype("arial.ttf", text_x)  # Adjust font size as needed

        # Add text to image
        draw.text((text_x, text_y), text, font=font, fill="black")  # Adjust fill color as needed

        # Append the modified image to the word_images list
        word_images.append(index_image)
    
    return word_images

# grid_images = processor.grid_images
# word_images = add_words_to_images(grid_images = grid_images, grid_letters=grid_letters)
# merged_image = processor.remerge_grid_images(word_images,grid_size=32,
#                               square_height=processor.square_height,
#                               square_width=processor.square_height)

# display(merged_image)

word_images = add_words_to_images(grid_images = processor.grid_images[0:1], grid_letters=['k'])
display(word_images[0])

AttributeError: 'ImageGridProcessor' object has no attribute 'grid_images'

In [ ]:
word_images = add_words_to_images(grid_images = grid_images[0:1], grid_letters=['k'])
display(word_images[0])

NameError: name 'grid_images' is not defined

In [ ]:
from PIL import Image, ImageDraw, ImageFont

def add_words_to_images(grid_images, grid_letters):
    word_images = []
    
    for index_image, index_letter in zip(grid_images, grid_letters):
        # Create a draw object
        draw = ImageDraw.Draw(index_image)

        # Define the text and font
        text = index_letter
        font = ImageFont.truetype("arial.ttf", 100)  # Adjust font size as needed

        # Calculate text size and position
        text_width, text_height = draw.textsize(text, font=font)
        image_width, image_height = index_image.size
        text_x = (image_width - text_width) // 2
        text_y = (image_height - text_height) // 2

        # Add text to image
        draw.text((text_x, text_y), text, font=font, fill="white")  # Adjust fill color as needed

        # Append the modified image to the word_images list
        word_images.append(index_image)
    
    return word_images
